# **1. Load Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# read in the extracted text file      
with open("/content/drive/MyDrive/word-embedding-creation/input/text8", "r") as f:
    text = f.read()

# print out the first 100 characters
print(text[:100])

 anarchism originated as a term of abuse first used against early working class radicals including t


# **2. Preprocess Data**

Because the corpus is very **large**, if we take all vocabulary the appear in the corpus into consideration, our vocab dictionary grows up to 250,000 words. This cause our models to be unreasonably large and take a large amount of time to train. On the other hand, there are many words that appear only several times in the corpus (< 10); thus, we will not be able to obtain good dense representations for it. Therefore, it is essential for use to only choose our corpus to be the 30000 most frequent words.

In [ ]:
from collections import Counter
corpus = text.split()
c = Counter(corpus)
most_30000 = c.most_common(30000)
vocab_dictionary = [i[0] for i in most_30000]
print(vocab_dictionary[:100])

['the', 'of', 'and', 'one', 'in', 'a', 'to', 'zero', 'nine', 'two', 'is', 'as', 'eight', 'for', 's', 'five', 'three', 'was', 'by', 'that', 'four', 'six', 'seven', 'with', 'on', 'are', 'it', 'from', 'or', 'his', 'an', 'be', 'this', 'which', 'at', 'he', 'also', 'not', 'have', 'were', 'has', 'but', 'other', 'their', 'its', 'first', 'they', 'some', 'had', 'all', 'more', 'most', 'can', 'been', 'such', 'many', 'who', 'new', 'used', 'there', 'after', 'when', 'into', 'american', 'time', 'these', 'only', 'see', 'may', 'than', 'world', 'i', 'b', 'would', 'd', 'no', 'however', 'between', 'about', 'over', 'years', 'states', 'people', 'war', 'during', 'united', 'known', 'if', 'called', 'use', 'th', 'system', 'often', 'state', 'so', 'history', 'will', 'up', 'while', 'where']


In [ ]:
int_to_vocab = {ii + 1: word for ii, word in enumerate(vocab_dictionary)}
int_to_vocab[0] = '<OOV>'
vocab_to_int = {word: ii + 1 for ii, word in enumerate(vocab_dictionary)}
vocab_to_int['<OOV>'] = 0

In [ ]:
int_sequeces = []
for word in corpus:
    if word in vocab_to_int:
        int_sequeces.append(vocab_to_int[word])
    else:
        int_sequeces.append(0)
# int_words = [vocab_to_int[word] for word in words]

print(int_sequeces[:30])
print(len(int_sequeces))

[5234, 3081, 12, 6, 195, 2, 3134, 46, 59, 156, 128, 742, 477, 10572, 134, 1, 27350, 2, 1, 103, 855, 3, 1, 15068, 0, 2, 1, 151, 855, 3581]
17005207


## Subsampling

[(Mikolov et al., 2013)](https://proceedings.neurips.cc/paper/2013/file/9aa42b31882ec039965f3c4923ce901b-Paper.pdf)
In very large corpora, the most frequent words can easily occur hundreds of millions of times (e.g., “in”, “the”, and “a”). Such words usually provide less information value than the rare words. For example, while the Skip-gram model benefits from observing the co-occurrences of “France” and “Paris”, it benefits much less from observing the frequent co-occurrences of “France” and “the”, as
nearly every word co-occurs frequently within a sentence with “the”. This idea can also be applied in the opposite direction; the vector representations of frequent words do not change significantly after training on several million examples.
To counter the imbalance between the rare and frequent words, we used a simple subsampling approach: each word $w_i$ in the training set is discarded with probability computed by the formula

$$ P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}} $$

where $f(w_i)$ is the frequency of word $w_i$ and $t$ is a chosen threshold, typically around $10^{−5}$. We chose this subsampling formula because it aggressively subsamples words whose frequency is greater than t while preserving the ranking of the frequencies. Although this subsampling formula was chosen heuristically, we found it to work well in practice. It accelerates learning and even significantly improves the accuracy of the learned vectors of the rare words, as will be shown in the following sections.

In [ ]:
from collections import Counter
import random
import numpy as np

threshold = 1/10**5                              # t in the formula

word_counts = Counter(int_sequeces)             # for calculating $f(w_i)$
total_count = len(int_sequeces)                 
freqs = {word: count/total_count for word, count in word_counts.items()}        # dictionary of every word's frequency

p_drop = {word: 1 - np.sqrt(threshold/freqs[word]) for word in word_counts}     # dictionary of every word's probability of discarding
train_words = [word for word in int_sequeces if random.random() < (1 - p_drop[word])]

print(train_words[:30])

[742, 10572, 27350, 15068, 855, 3581, 195, 5, 10713, 1325, 105, 59, 2732, 363, 3673, 7089, 28, 2878, 603, 1135, 8984, 32, 4148, 6438, 5234, 345, 4861, 6754, 7574, 11065]


## 3. Making batches

According to Mikolov et al., 2013

"Since the more distant words are usually less related to the current word than those close to it, we give less weight to the distant words by sampling less from those words in our training examples... If we choose $C=5$, for each training word we will select randomly a number $R$ in range $[1 : C]$, and then use $R$ words from history and  words from the future of the current word as correct labels."


In [ ]:
def get_labels(words, idx, window_size=5):
    ''' Get a list of context words (which is also target words in training phase) in a window around a center word. '''
    
    R = np.random.randint(1, window_size+1)
    start = idx - R if (idx - R) > 0 else 0
    stop = idx + R
    labels = [word for word in words[start:idx] + words[idx+1:stop+1] if word != 0]
    
    return labels

In [ ]:
import torch
def create_iterative_batches(words, batch_size, window_size=3):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    n_batches = len(words)//batch_size
    
    # we discard some last tokens that are not in any batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_labels(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y
    

---
## Validation

Here we use cosine similarity to track current performances of training models.

$$
\mathrm{similarity} = \cos(\theta) = \frac{\vec{a} \cdot \vec{b}}{|\vec{a}||\vec{b}|}
$$


In [ ]:
def cosine_similarity(embedding, valid_size=16, valid_window=100, device='cpu'):
    """ Returns the cosine similarity of validation words with words in the embedding matrix.
        Here, embedding should be a PyTorch embedding module.
    """
    
    # Here we're calculating the cosine similarity between some random words and 
    # our embedding vectors. With the similarities, we can look at what words are
    # close to our random words.
    
    # sim = (a . b) / |a||b|
    
    embed_vectors = embedding.weight
    
    # magnitude of embedding vectors, |b|
    magnitudes = embed_vectors.pow(2).sum(dim=1).sqrt().unsqueeze(0)
    
    # pick N words from our ranges (0,window) and (1000,1000+window). lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples,
                               random.sample(range(1000,1000+valid_window), valid_size//2))
    valid_examples = torch.LongTensor(valid_examples).to(device)
    
    valid_vectors = embedding(valid_examples)
    similarities = torch.mm(valid_vectors, embed_vectors.t())/magnitudes
        
    return valid_examples, similarities

## SkipGram

In [ ]:
import torch
from torch import nn
import torch.optim as optim
class SkipGram(nn.Module):
    def __init__(self, n_vocab, n_embed):
        super().__init__()
        
        self.embed = nn.Embedding(n_vocab, n_embed)
        self.output = nn.Linear(n_embed, n_vocab)
        self.log_softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        x = self.embed(x)
        scores = self.output(x)
        log_ps = self.log_softmax(scores)
        
        return log_ps

In [ ]:
# check if GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

embedding_dim=100 

model = SkipGram(len(vocab_to_int), embedding_dim).to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

# With pytorch, we need to manually print the training stats to keep track of the progress
print_every = 2500
steps = 0
epochs = 15

running_loss = 0.0
for e in range(epochs):
    # get input and target batches
    for inputs, targets in create_iterative_batches(train_words, 1024):
        steps += 1
        inputs, targets = torch.LongTensor(inputs), torch.LongTensor(targets)
        inputs, targets = inputs.to(device), targets.to(device)
        
        log_ps = model(inputs)
        loss = criterion(log_ps, targets)
        running_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if steps % print_every == 0:                  
            # getting examples and similarities      
            valid_examples, valid_similarities = cosine_similarity(model.embed, device=device)
            _, closest_idxs = valid_similarities.topk(6) # topk highest similarities
            valid_examples, closest_idxs = valid_examples.to('cpu'), closest_idxs.to('cpu')
            
            print(steps, running_loss/2500)
            running_loss = 0.0

            for ii, valid_idx in enumerate(valid_examples):
                closest_words = [int_to_vocab[idx.item()] for idx in closest_idxs[ii]][1:]
                print(int_to_vocab[valid_idx.item()] + " | " + ', '.join(closest_words))
            print("...")

2500 9.861569799423219
called | stables, characteristic, zeus, violated, hexagonal
people | turing, indo, baptized, relieving, religions
has | comprehend, moors, tripolitania, pygmy, colonial
this | resource, tutorials, lifeforms, regulations, deism
with | auchinleck, spectator, far, ouest, retain
while | requirements, tao, livery, conspirators, macintoshes
states | aroma, hardcover, mehmet, shetland, trappings
war | diocletian, alexandra, babur, zedong, unintended
bible | autopsy, morrison, canon, testament, eclectic
recorded | voters, misunderstood, millet, winter, guaranteeing
defense | fief, season, pants, aeolus, monarchy
frac | decoding, solution, deuce, claim, vendors
professional | chan, heinrich, pyle, ringing, musketeers
freedom | underway, mature, restructured, determining, animist
smith | matteo, mansfield, rodrigo, summaries, stamp
additional | monteverdi, frontal, retained, solstice, arbor
...
5000 9.555713206100464
and | finally, gems, stadiums, zbigniew, goshen
is | cor

In [ ]:
import numpy as np
numpy_embedding = model.embed.weight.detach().to('cpu').numpy()

In [ ]:
numpy_embedding.shape

(30001, 100)

In [ ]:
import os
import numpy as np
import random
import torch
def find_cosine_similarity(vector_matrix, v):
    """
    vector_matrix: pytorch tensor (matrix) (word representations for the whole vocabulary dictionary)
    v: pytorch tensor (vector we need to find best consine similarity score for)
    """
    # find the dot product between u and v 
    dot = torch.matmul(vector_matrix, torch.transpose(v.unsqueeze(0),0,1))
    # find the L2 norm of u 
    norm_u = torch.sqrt(torch.sum(vector_matrix**2, dim=1))
    # Compute the L2 norm of v
    norm_v = torch.sqrt(torch.sum((v**2).unsqueeze(0), dim=1))
    # Compute the cosine similarity
    norm_u = norm_u.unsqueeze(1)
    cosine_sim = dot/norm_u/norm_v
    
    return cosine_sim

def find_analogy(word_a, word_b, word_c, words_vocabulary, vector_matrix):
    """
    Parameters:

        word_a, word_b, word_c: Python string (The three input words of word analogy test)
        words_vocabulary: Python dictionary (The dictionary mapping vocaburay into index)
        vector_matrix: Pytorch tensor(matrix) (word representations for the whole vocabulary dictionary)
    Return:
    
        best_word: The best word predicted given pretrained representations
    """

    # convert words to lower case
    word_a = word_a.lower()
    word_b = word_b.lower()
    word_c = word_c.lower()
    
    # Find the word embeddings for word_a, word_b, word_c
    word_a = words_vocabulary[word_a]
    word_b = words_vocabulary[word_b]
    word_c = words_vocabulary[word_c]
    e_a, e_b, e_c = vector_matrix[word_a], vector_matrix[word_b], vector_matrix[word_c]

    # Compute cosine similarity between the vectors u and v
    # u:(e_b - e_a) 
    # v:((w's vector representation) - e_c)

    # Calculate all similarity score over the whole dictionary
    cosine_sim = find_cosine_similarity(vector_matrix, e_b - e_a + e_c)
    
    # eliminate word_a, word_b, word_c from options for prediction
    cosine_sim[word_a] = -float("Inf")
    cosine_sim[word_b] = -float("Inf")
    cosine_sim[word_c] = -float("Inf")
    cosine_sim[0] = -float("Inf")

    # use argmax to find word with highest similarity score
    max_index = torch.argmax(cosine_sim)
    max_index = int(max_index)
    for word in words_vocabulary:
        if words_vocabulary[word] == max_index:
            best_word = word
            break
    return best_word

# for taking input from the user and doing word analogy task on that
def word_analogy_test():
    # Load pretrained word representations from txt file
    words_vocabulary, vector_matrix = vocab_to_int, torch.from_numpy(numpy_embedding)
    inp = []
    out = []

    # Get testing samples from txt file
    with open("drive/MyDrive/NLP-2021/Project1-NLP/WordAnalogy-GLoVe/data/data.txt") as f:
        lines = f.readlines()
        for line in lines:
            if line[0].isalpha():
                line = line[:-1]
                data = line.split()
                for i in range(len(data)):
                    data[i] = data[i].lower()
                inp.append(data[:3])
                out.append(data[3])
    f.close()
    # Count correct tests
    correct = 0
    total = 0

    for i in range(len(inp)):
      # because we only use 30,000 most frequent words in our datasets, there are cases that words in samples are not in our vocabulary dictionary
        try:
            best_pick = find_analogy(*inp[i], words_vocabulary, vector_matrix)
            if out[i].lower() not in list(words_vocabulary.keys()):
                pass
            if best_pick == out[i].lower():
                correct += 1
            total += 1
        except:
            pass
    print("Correct:", correct)
    print("Total:", total)
word_analogy_test()

Correct: 2808
Total: 13537
